In [1]:
!pip install langchain-google-genai  -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 515.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


# envio do arquivo como contexto

In [2]:
import google.generativeai as genai
import time
import json
import os

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List



from google.colab import userdata
apiKey = userdata.get('GEMINI_API_KEY')
print("Chave API carregada do Colab Userdata.")

genai.configure(api_key=apiKey)



/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


Chave API carregada do Colab Userdata.


In [3]:
class PerguntaResposta(BaseModel):
    pergunta: str = Field(description="Uma pergunta sobre astronomia.")
    resposta: str = Field(description="Uma resposta detalhada e educativa para a pergunta.")

arquivo_json = "perguntas_astronomia_langchain.json"
perguntas_existentes_lista = []
dados_existentes = []

if os.path.exists(arquivo_json):
    try:
        with open(arquivo_json, "r", encoding="utf-8") as arquivo:
            dados_existentes = json.load(arquivo)
        if isinstance(dados_existentes, list):
            perguntas_existentes_lista = [item.get("pergunta", "") for item in dados_existentes if item.get("pergunta")]
            print(f"Carregadas {len(perguntas_existentes_lista)} perguntas existentes do arquivo.")
        else:
            print(f"Aviso: Arquivo '{arquivo_json}' não contém uma lista JSON válida. Começando sem histórico.")
            dados_existentes = []
    except json.JSONDecodeError:
        print(f"Aviso: Não foi possível decodificar JSON do arquivo existente '{arquivo_json}'. Começando sem histórico.")
        dados_existentes = []
    except Exception as e:
         print(f"Erro ao ler o arquivo existente '{arquivo_json}': {e}. Começando sem histórico.")
         dados_existentes = []
else:
    print(f"Arquivo '{arquivo_json}' não encontrado. Gerando as primeiras perguntas.")

Carregadas 241 perguntas existentes do arquivo.


In [16]:
MAX_PERGUNTAS_NO_PROMPT = 300
perguntas_para_prompt = "\n".join(f"- {p}" for p in perguntas_existentes_lista[:MAX_PERGUNTAS_NO_PROMPT])
if not perguntas_para_prompt:
    perguntas_para_prompt = "Nenhuma pergunta existente fornecida."

#(model_name="models/gemini-2.5-pro-exp-03-25") # models/gemini-2.5-pro-preview-03-25     models/gemini-2.0-flash

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro-exp-03-25",
    google_api_key=apiKey,
    temperature=0.5,
    request_options={"timeout": 600}
)

texto_template_prompt = """
**Role:**
Você é um assistente especializado em gerar dados sintéticos realistas sobre astronomia, especificamente perguntas e respostas educativas destinadas ao público geral interessado em astronomia.

**Contexto:**
O objetivo é expandir uma base de conhecimento existente para treinar modelos de linguagem. Queremos adicionar perguntas novas e interessantes que complementem as que já temos.

**Perguntas Existentes:**
Aqui estão algumas perguntas que já existem na nossa base de dados:
{perguntas_existentes}```

Instructions:
Gere um conjunto de 10 NOVAS perguntas e respostas sobre astronomia, planetas, luas, astronáutica ou exploração espacial.
CRÍTICO: As perguntas geradas devem ser DIFERENTES das perguntas listadas na seção "Perguntas Existentes". Evite repetir os mesmos tópicos ou perguntas formuladas de maneira muito similar.
Tente gerar perguntas que COMPLEMENTEM os tópicos existentes, explorando facetas diferentes ou relacionadas aos temas gerais da astronomia e exploração espacial.
As perguntas devem variar em complexidade.
As respostas devem ser educativas, precisas e fáceis de entender.
Formato OBRIGATÓRIO: Gere o conteúdo APENAS no formato JSON. O JSON deve ser uma lista de objetos, onde cada objeto tem as chaves "pergunta" e "resposta". Não inclua nenhuma introdução, explicação ou markdown json antes ou depois do JSON puro.
Exemplo de Saída Esperada (APENAS O JSON):
[
{{
"pergunta": "Como as auroras boreais são formadas?",
"resposta": "As auroras são causadas por partículas carregadas (elétrons e prótons) do Sol, trazidas pelo vento solar, que colidem com gases na alta atmosfera da Terra (principalmente oxigênio e nitrogênio). Essas colisões excitam os átomos dos gases, que liberam energia na forma de luz colorida."
}},
{{
"pergunta": "O que é a matéria escura e por que os cientistas acreditam que ela existe?",
"resposta": "Matéria escura é uma forma hipotética de matéria que não interage com a luz (é invisível), mas exerce gravidade. Os cientistas inferem sua existência observando seus efeitos gravitacionais em galáxias e aglomerados de galáxias, como a velocidade de rotação das galáxias, que é maior do que seria esperado apenas com a matéria visível."
}}
]
Tarefa:
Gere 10 NOVAS perguntas e respostas sobre estrelas, a Lua, a Terra, o sistema solar, astronomia , os planetas em detalhes, luas, astronáutica ou exploração espacial que NÃO estão na lista fornecida, seguindo ESTRITAMENTE o formato JSON de lista de objetos.

"""

In [17]:
prompt = ChatPromptTemplate.from_template(texto_template_prompt)
parser = JsonOutputParser(pydantic_object=PerguntaResposta)
chain = prompt | llm | parser

In [18]:
prompt

ChatPromptTemplate(input_variables=['perguntas_existentes'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['perguntas_existentes'], input_types={}, partial_variables={}, template='\n**Role:**\nVocê é um assistente especializado em gerar dados sintéticos realistas sobre astronomia, especificamente perguntas e respostas educativas destinadas ao público geral interessado em astronomia.\n\n**Contexto:**\nO objetivo é expandir uma base de conhecimento existente para treinar modelos de linguagem. Queremos adicionar perguntas novas e interessantes que complementem as que já temos.\n\n**Perguntas Existentes:**\nAqui estão algumas perguntas que já existem na nossa base de dados:\n{perguntas_existentes}```\n\nInstructions:\nGere um conjunto de 10 NOVAS perguntas e respostas sobre astronomia, planetas, luas, astronáutica ou exploração espacial.\nCRÍTICO: As perguntas geradas devem ser DIFERENTES das perguntas listadas na seção "P

In [26]:
print("Realizando requisição para o LLM via Langchain com contexto de perguntas existentes...")
novos_dados = []

try:
  start_time = time.time()
  dados_resposta = chain.invoke({"perguntas_existentes": perguntas_para_prompt})
  end_time = time.time()

  print(f"Chamada ao LLM levou {end_time - start_time:.2f} segundos.")

  if isinstance(dados_resposta, list):
     novos_dados = dados_resposta
     perguntas_novas_set = {item.get("pergunta", "").strip().lower() for item in novos_dados if item.get("pergunta")}
     perguntas_existentes_set = {p.strip().lower() for p in perguntas_existentes_lista}
     duplicatas_geradas = perguntas_novas_set.intersection(perguntas_existentes_set)
     if duplicatas_geradas:
         print(f"Aviso: O LLM gerou {len(duplicatas_geradas)} perguntas que já existiam, apesar da instrução.")

  else:
    print("Aviso: Os dados parseados não são uma lista. Verificando a saída do LLM e o parser.")
    if dados_resposta:
         if isinstance(dados_resposta, dict) and "pergunta" in dados_resposta:
             novos_dados = [dados_resposta]
             print("Aviso: Tratado como lista de um item.")


except Exception as e:
  print(f"Erro durante a execução da cadeia Langchain ou parsing: {e}")



Realizando requisição para o LLM via Langchain com contexto de perguntas existentes...
Chamada ao LLM levou 45.33 segundos.


In [ ]:
"""try:
  raw_output = (prompt | llm).invoke({"perguntas_existentes": perguntas_para_prompt})
  print("\n--- Saída Bruta do LLM ---")
  print(raw_output.content)
  print("------------------------\n")
except Exception as llm_e:
  print(f"Não foi possível obter a saída bruta do LLM: {llm_e}")
  novos_dados = []
  print(f"Novos dados brutos recebidos (antes de salvar): {novos_dados}")"""

In [27]:
print(f"Novos dados brutos: {novos_dados}")

Novos dados brutos: [{'pergunta': 'O que são erupções solares (solar flares) e como elas podem afetar a Terra?', 'resposta': 'Erupções solares são explosões intensas de radiação na superfície do Sol, liberando energia e partículas carregadas. Quando direcionadas à Terra, essas partículas podem interagir com nosso campo magnético e atmosfera, causando auroras mais intensas, interferindo em comunicações de rádio, redes elétricas e satélites, um fenômeno conhecido como clima espacial.'}, {'pergunta': 'De onde vêm os elementos químicos mais pesados que o hidrogênio e o hélio, como o carbono e o oxigênio que formam a vida?', 'resposta': 'Esses elementos são criados no interior das estrelas através da fusão nuclear (nucleossíntese estelar). Estrelas como o Sol fundem hidrogênio em hélio. Estrelas mais massivas podem fundir elementos mais pesados, como carbono, oxigênio, até ferro. Elementos ainda mais pesados que o ferro são formados principalmente durante eventos cósmicos energéticos, como 

In [28]:
arquivo_json = "perguntas_astronomia_langchain.json"
perguntas_existentes_lista = []
dados_existentes = []

if os.path.exists(arquivo_json):
    try:
        with open(arquivo_json, "r", encoding="utf-8") as arquivo:
            dados_existentes = json.load(arquivo)
        if isinstance(dados_existentes, list):
            perguntas_existentes_lista = [item.get("pergunta", "") for item in dados_existentes if item.get("pergunta")]
            print(f"Carregadas {len(perguntas_existentes_lista)} perguntas existentes do arquivo.")
        else:
            print(f"Aviso: Arquivo '{arquivo_json}' não contém uma lista JSON válida. Começando sem histórico.")
            dados_existentes = []
    except json.JSONDecodeError:
        print(f"Aviso: Não foi possível decodificar JSON do arquivo existente '{arquivo_json}'. Começando sem histórico.")
        dados_existentes = []
    except Exception as e:
         print(f"Erro ao ler o arquivo existente '{arquivo_json}': {e}. Começando sem histórico.")
         dados_existentes = []
else:
    print(f"Arquivo '{arquivo_json}' não encontrado. Gerando as primeiras perguntas.")

Carregadas 271 perguntas existentes do arquivo.


In [29]:

print(f"Gerados e parseados com sucesso {len(novos_dados)} novos pares de P&R.")
dados_atualizados = dados_existentes + novos_dados
print(f"Total após combinação (pode conter duplicatas): {len(dados_atualizados)} perguntas")


try:
        with open(arquivo_json, "w", encoding="utf-8") as arquivo:
            json.dump(dados_atualizados, arquivo, ensure_ascii=False, indent=2)

        print(f"Arquivo '{arquivo_json}' atualizado com sucesso com {len(dados_atualizados)} perguntas (incluindo possíveis duplicatas)!")
        if dados_atualizados:
             # Mostra a última pergunta adicionada, que pode ser uma duplicata
             print(f"Última pergunta adicionada: {dados_atualizados[-1].get('pergunta', 'N/A')}")
except Exception as e:
        print(f"Erro ao escrever no arquivo JSON: {e}")



Gerados e parseados com sucesso 10 novos pares de P&R.
Total após combinação (pode conter duplicatas): 281 perguntas
Arquivo 'perguntas_astronomia_langchain.json' atualizado com sucesso com 281 perguntas (incluindo possíveis duplicatas)!
Última pergunta adicionada: Qual a diferença fundamental entre uma nebulosa de emissão, uma nebulosa de reflexão e uma nebulosa escura?


In [30]:
len(dados_atualizados)

281